In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.style.use('bmh')

In [3]:
import numpy as np

import torch
import math
import random

from tqdm import tqdm_notebook
from boltons.iterutils import pairwise
from scipy.stats import kendalltau
from itertools import permutations

from sent_order.models.kt_regression import SentenceEncoder, Regressor, Corpus, Batch
from sent_order.perms import sample_uniform_perms

In [4]:
sent_encoder = torch.load(
    '../../data/models/new/kt-reg-clamp/sent_encoder.350.bin',
    map_location={'cuda:0': 'cpu'},
)

In [5]:
regressor = torch.load(
    '../../data/models/new/kt-reg-clamp/regressor.350.bin',
    map_location={'cuda:0': 'cpu'},
)

In [6]:
train = Corpus('../../data/dev.json/', 10000)

100%|██████████| 10000/10000 [00:01<00:00, 7423.25it/s]


In [13]:
grafs = [g for g in train.grafs if len(g.sentences) == 2]

In [14]:
len(grafs)

878

In [15]:
def predict(graf):
    
    sents = sent_encoder(graf.sentence_variables())
    
    perms = list(permutations(range(len(sents))))
    
    x = torch.stack([
        sents[torch.LongTensor(perm)]
        for perm in perms
    ])
    
    scores = regressor(x)
    
    return perms[torch.min(scores, 0)[1].data[0]]

In [16]:
kts = []
for g in tqdm_notebook(grafs):
    pred = predict(g)
    kt = kendalltau(pred, range(len(pred))).correlation
    kts.append(kt)

HBox(children=(IntProgress(value=0, max=878), HTML(value='')))

/Users/dclure/Projects/plot-ordering/env/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


In [11]:
np.mean(kts)

0.80465502909393194

In [12]:
kts.count(1) / len(kts)

0.7481296758104738